## 内積計算を使って、色に当てられた形容詞をもとにその画像に当てはまる形容詞を、以下二つの数値パターンで算出してみた
- 画像に含まれる色(rgb値)の平均値
- 画像に含まれる色(rgb値)の最頻値

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import numpy as np
import os

# filepathで指定した画像ファイルを読み込み、その画像のRGB値の平均をNumpyのArrayで【行ベクトルで】返す関数　ext_mean_rgb(filepath)
def ext_mean_rgb(filepath):
  image = np.array(Image.open(filepath).convert('RGB')).reshape(-1,3)
  rgb_mean = np.array([np.mean(image[:,0]),np.mean(image[:,1]),np.mean(image[:,2])])
  return rgb_mean

In [ ]:
import numpy as np
from scipy.spatial import distance

# RGB値をnumpy array、qcolor=np.array([r,g,b])で与えられると[赤,橙,黄,緑,青,紫,ピンク,白,グレー,黒]の重みを成分とする10次元ベクトルが出力される関数gen_color_vec(qcolor)
def gen_color_vec(qcolor):
  colorvec=np.array([])
  palette=np.array(
      [
       [255,0,0], #赤
      [255,102,0],  #橙
      [255,255,0],  #黄
      [0,128,0],  #緑
      [0,0,255],  #青
      [128,0,128],  #紫
      [255,0,255],  #ピンク
      [255,255,255],  #白
      [128,128,128],  #グレー
      [0,0,0] #黒
      ])
  for col in palette:
    colorvec=np.append(colorvec,distance.euclidean(col,qcolor))
  colorvec=1-colorvec/np.linalg.norm(colorvec,np.inf)
  return colorvec

In [ ]:
import pandas as pd

#印象語と色のデータ
df = pd.read_csv('datas/color_image.csv', encoding='shift-jis')
df

,Unnamed: 0,情熱的,強い,派手,元気,楽しい,かわいい,危険,落ち着いた,健康的,穏やかな,冷静,冷たい,気高い,神秘的,甘やか,自由な,純粋な,孤独な,寂しい,かっこいい
0,red,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,orange,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,yellow,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,green,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
4,blue,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0
5,purple,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0
6,pink,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,white,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,0,0,0
8,gray,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1
9,black,0,0,0,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1


In [ ]:
#印象語をリストで取り出す
im = df.columns.values
im = np.delete(im, 0)
im

array(['情熱的', '強い', '派手', '元気', '楽しい', 'かわいい', '危険', '落ち着いた', '健康的',
       '穏やかな', '冷静', '冷たい', '気高い', '神秘的', '甘やか', '自由な', '純粋な', '孤独な',
       '寂しい', 'かっこいい'], dtype=object)

In [ ]:
#行列のみ取り出す
color_image = df.iloc[:, 1:].values
color_image

array([[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1]])

<h1>(パターン1)平均値で計算</h1>

In [ ]:
def result(x):
  #印象語行列と重みを掛け算
  image_weight = np.dot(color_image, x)
  #列ごとに足し算し1行ベクトルにする
  image_weight = np.sum(image_weight, axis = 0)
  #最大値のインデックスを取得
  max_im = np.where(image_weight == image_weight.max())

  #最大値のインデックスから印象語を抽出し出力
  for i in max_im:
    print(im[i])

In [ ]:
#使った画像と平均rgb値と印象語
filepath='画像へのパス'
rgb_mean = ext_mean_rgb(filepath)
x=gen_color_vec(rgb_mean)
print("[r, g, b]:", rgb_mean)
result(x)
Image.open(filepath)

In [ ]:
#使った画像と平均rgb値と印象語
filepath='画像へのパス'
rgb_mean = ext_mean_rgb(filepath)
x=gen_color_vec(rgb_mean).reshape(-1,1)
print("[r, g, b]:", rgb_mean)
result(x)

Image.open(filepath)

<h1>(パターン2)最頻値で計算</h1>

In [ ]:
def ext_mode_rgb(filepath):
  image = np.array(Image.open(filepath).convert('RGB')).reshape(-1,3)
  #rgb値が出てくる回数を記録し、その記録の最大値を持つrgb値を出力
  pixel_sum = []
  pixel_colors = []
  for col in image:
    col = col.tolist()
    if col in pixel_colors:
      pixel_sum[pixel_colors.index(col)] += 1
    else:
      pixel_colors.append(col)
      pixel_sum.append(1)

  rgb_mode = pixel_colors[pixel_sum.index(max(pixel_sum))]
  rgb_mode = np.array(rgb_mode) 

  return rgb_mode

In [ ]:
filepath='画像へのパス'
rgb_mode = ext_mode_rgb(filepath)
x=gen_color_vec(rgb_mode).reshape(-1,1)
print("[r, g, b]:", rgb_mode)
result(x)
Image.open(filepath)

In [ ]:
filepath='画像へのパス'
rgb_mode = ext_mode_rgb(filepath)
x=gen_color_vec(rgb_mode).reshape(-1,1)
print("[r, g, b]:", rgb_mode)
result(x)
Image.open(filepath)